In [5]:
import pandas as pd
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import LSTM, Dense, Embedding, Dropout, Input, Concatenate
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.models import load_model
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [19]:
third_model = load_model('/content/drive/MyDrive/Final Election Project/model_2013.h5')
df_2018 = pd.read_csv('/content/drive/MyDrive/Final Election Project/Complete 2018.csv')

In [20]:
len(df_2018['constituency_number'].unique())

272

In [21]:
desire_col = ['constituency_number','province','candidate_name','candidate_party','voter_reg','outcome']
data = df_2018[desire_col]

In [22]:
max_candidate_name_length = 40
max_candidate_party_length = 40

In [23]:
def tokenize_and_pad(data, max_len):
    sequences = pad_sequences(data, maxlen=max_len, padding='post')
    return sequences.tolist()

In [24]:
data['candidate_name'] = data.groupby('constituency_number')['candidate_name'].transform(lambda x: x.astype(str).tolist())
data['candidate_party'] = data.groupby('constituency_number')['candidate_party'].transform(lambda x: x.astype(str).tolist())

# Extract unique constituencies
constituencies = data['constituency_number'].unique()

<ipython-input-24-df51f43c78f4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['candidate_name'] = data.groupby('constituency_number')['candidate_name'].transform(lambda x: x.astype(str).tolist())
<ipython-input-24-df51f43c78f4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['candidate_party'] = data.groupby('constituency_number')['candidate_party'].transform(lambda x: x.astype(str).tolist())


In [25]:
all_features, all_labels = [], []

for constituency_number in constituencies:
    group = data[data['constituency_number'] == constituency_number]

    features = group[['province', 'candidate_name', 'candidate_party', 'voter_reg' ]]
    labels = group['outcome'].astype('category').cat.codes

    all_features.append(features)
    all_labels.append(labels)

processed_data = pd.concat(all_features, ignore_index=True)
processed_data['outcome'] = pd.concat(all_labels, ignore_index=True)

In [26]:
# Convert text columns to sequences
tok = Tokenizer(num_words=10000)
tok.fit_on_texts(processed_data['candidate_name'].astype(str))
processed_data['candidate_name_padded'] = tok.texts_to_sequences(processed_data['candidate_name'].astype(str))

tok.fit_on_texts(processed_data['candidate_party'].astype(str))
processed_data['candidate_party_padded'] = tok.texts_to_sequences(processed_data['candidate_party'].astype(str))

In [27]:
X_train, X_test, y_train, y_test = train_test_split(processed_data[['province', 'candidate_name_padded', 'candidate_party_padded','voter_reg']], processed_data['outcome'], test_size=0.2, random_state=42)

In [34]:
from keras.layers import Bidirectional, Reshape

model4 = Sequential()
model4.add(third_model)
model4.add(Reshape((-1, 1)))
model4.add(LSTM(64, return_sequences=True))
model4.add(Dropout(0.2))
model4.add(LSTM(32, return_sequences=True))
model4.add(LSTM(32))
model4.add(Dense(16, activation='relu'))
model4.add(Dense(1, activation='tanh'))
model4.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [30]:
X_train_padded = pad_sequences(X_train['candidate_name_padded'].tolist(), maxlen=max_candidate_name_length, padding='post', truncating='post', dtype='float32')

# Assuming max_candidate_name_length is the length of your padded sequences
timesteps = max_candidate_name_length
features = 1  # Assuming each element in the sequence is a single feature

X_train_padded = X_train_padded.reshape((X_train_padded.shape[0], timesteps, features))

In [35]:
model4.fit(X_train_padded, y_train, epochs=10, validation_split=0.2)

Epoch 1/10
65/65 [==============================] - 49s 217ms/step - loss: 1.3689 - accuracy: 0.9113 - val_loss: 1.0463 - val_accuracy: 0.9283
Epoch 2/10
65/65 [==============================] - 8s 122ms/step - loss: 1.3689 - accuracy: 0.9113 - val_loss: 1.0463 - val_accuracy: 0.9283
Epoch 3/10
65/65 [==============================] - 7s 101ms/step - loss: 1.3689 - accuracy: 0.9113 - val_loss: 1.0463 - val_accuracy: 0.9283
Epoch 4/10
65/65 [==============================] - 8s 117ms/step - loss: 1.3689 - accuracy: 0.9113 - val_loss: 1.0463 - val_accuracy: 0.9283
Epoch 5/10
65/65 [==============================] - 7s 107ms/step - loss: 1.3689 - accuracy: 0.9113 - val_loss: 1.0463 - val_accuracy: 0.9283
Epoch 6/10
65/65 [==============================] - 7s 106ms/step - loss: 1.3689 - accuracy: 0.9113 - val_loss: 1.0463 - val_accuracy: 0.9283
Epoch 7/10
65/65 [==============================] - 7s 108ms/step - loss: 1.3689 - accuracy: 0.9113 - val_loss: 1.0463 - val_accuracy: 0.9283
Epoch

In [36]:
X_test_padded = pad_sequences(X_test['candidate_name_padded'].tolist(), maxlen=max_candidate_name_length, padding='post', truncating='post', dtype='float32')
X_test_padded = X_test_padded.reshape((X_test_padded.shape[0], timesteps, features))

# Evaluate the model on the test set
evaluation = model4.evaluate(X_test_padded, y_test)
print("Test Loss:", evaluation[0])
print("Test Accuracy:", evaluation[1])

21/21 [==============================] - 0s 22ms/step - loss: 1.4349 - accuracy: 0.9070
Test Loss: 1.4348787069320679
Test Accuracy: 0.9069767594337463


In [38]:
data24 = pd.read_csv('/content/drive/MyDrive/Final Election Project/Election 2024.csv')

In [37]:
predictions_2018 = model4.predict(X_test_padded)
predictions_2018

21/21 [==============================] - 8s 22ms/step


array([[-0.01604877],
       [-0.01604877],
       [-0.01604877],
       [-0.01604877],
       [-0.01604877],
       [-0.01604877],
       [-0.01604877],
       [-0.01604877],
       [-0.01604877],
       [-0.01604877],
       [-0.01604877],
       [-0.01604877],
       [-0.01604877],
       [-0.01604877],
       [-0.01604877],
       [-0.01604877],
       [-0.01604877],
       [-0.01604877],
       [-0.01604877],
       [-0.01604877],
       [-0.01604877],
       [-0.01604877],
       [-0.01604877],
       [-0.01604877],
       [-0.01604877],
       [-0.01604877],
       [-0.01604877],
       [-0.01604877],
       [-0.01604877],
       [-0.01604877],
       [-0.01604877],
       [-0.01604877],
       [-0.01604877],
       [-0.01604877],
       [-0.01604877],
       [-0.01604877],
       [-0.01604877],
       [-0.01604877],
       [-0.01604877],
       [-0.01604877],
       [-0.01604877],
       [-0.01604877],
       [-0.01604877],
       [-0.01604877],
       [-0.01604877],
       [-0